In [98]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM,Flatten
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import RobustScaler, MaxAbsScaler
import tensorflow as tf

In [99]:
tf.set_random_seed(777)
 
# Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

# Min-Max scaling
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7) # 1e-7은 0으로 나누는 오류 예방차원
 
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

In [100]:
 
# 하이퍼파라미터
input_data_column_cnt = 6  # 입력데이터의 컬럼 개수(Variable 개수)
output_data_column_cnt = 1 # 결과데이터의 컬럼 개수
 
seq_length = 5            # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
 
epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01       # 학습률
 

In [101]:
df = pd.read_csv('samsung.csv',  encoding='cp949', thousands = ',')
del df['일자']
df.head()

,시가,고가,저가,종가,거래량
0,57800,58400,56400,56400,19749457
1,58800,58800,56800,57200,20821939
2,59100,59700,58800,59100,16446102
3,59400,59400,58300,58800,23664541
4,61800,61800,60700,60800,14916555


In [102]:
df = pd.DataFrame(df,columns=['시가','고가','저가','거래량','종가'])
df.head()

,시가,고가,저가,거래량,종가
0,57800,58400,56400,19749457,56400
1,58800,58800,56800,20821939,57200
2,59100,59700,58800,16446102,59100
3,59400,59400,58300,23664541,58800
4,61800,61800,60700,14916555,60800


In [103]:
# 부동소수점형으로 변환
stock_info = df.values[1:].astype(np.float)
stock_info[0]

array([   58800.,    58800.,    56800., 20821939.,    57200.])

In [104]:
x = stock_info[:,:-1]
x

array([[   58800.,    58800.,    56800., 20821939.],
       [   59100.,    59700.,    58800., 16446102.],
       [   59400.,    59400.,    58300., 23664541.],
       ...,
       [   52600.,    52800.,    50900., 16128305.],
       [   52600.,    53200.,    51900., 23104720.],
       [   53000.,    53900.,    51800., 39565391.]])

In [105]:
y = stock_info[:, [-1]]

In [106]:
y

array([[57200.],
       [59100.],
       [58800.],
       [60800.],
       [62300.],
       [61400.],
       [62400.],
       [61300.],
       [60700.],
       [59000.],
       [60000.],
       [60000.],
       [59500.],
       [58600.],
       [56800.],
       [55800.],
       [55500.],
       [55500.],
       [55200.],
       [55800.],
       [56500.],
       [55400.],
       [55000.],
       [55500.],
       [56000.],
       [56000.],
       [56300.],
       [56700.],
       [54700.],
       [54700.],
       [53300.],
       [51900.],
       [51500.],
       [51200.],
       [50400.],
       [49500.],
       [49450.],
       [49900.],
       [50400.],
       [50300.],
       [51300.],
       [52200.],
       [51800.],
       [51800.],
       [51600.],
       [51000.],
       [52000.],
       [53500.],
       [53500.],
       [53700.],
       [52800.],
       [52500.],
       [52600.],
       [51600.],
       [52100.],
       [52900.],
       [53300.],
       [52700.],
       [52300.

In [107]:
x_m = min_max_scaling(x)
x_m

array([[3.45918467e-04, 3.45918467e-04, 3.14399700e-04, 3.27560188e-01],
       [3.50646282e-04, 3.60101912e-04, 3.45918467e-04, 2.58599695e-01],
       [3.55374097e-04, 3.55374097e-04, 3.38038775e-04, 3.72357843e-01],
       ...,
       [2.48210290e-04, 2.51362166e-04, 2.21419338e-04, 2.53591410e-01],
       [2.48210290e-04, 2.57665920e-04, 2.37178721e-04, 3.63535409e-01],
       [2.54514043e-04, 2.68697488e-04, 2.35602783e-04, 6.22945435e-01]])

In [108]:
dataX = [] # 입력으로 사용될 Sequence Data
dataY = [] # 출력(타켓)으로 사용
 
for i in range(0, len(y) - seq_length):
    _x = x_m[i : i+seq_length]
    _y = y[i + seq_length] # 다음 나타날 주가(정답)
    if i is 0:
        print(_x, "->", _y) # 첫번째 행만 출력해 봄
    dataX.append(_x) # dataX 리스트에 추가
    dataY.append(_y) # dataY 리스트에 추가

[[3.45918467e-04 3.45918467e-04 3.14399700e-04 3.27560188e-01]
 [3.50646282e-04 3.60101912e-04 3.45918467e-04 2.58599695e-01]
 [3.55374097e-04 3.55374097e-04 3.38038775e-04 3.72357843e-01]
 [3.93196617e-04 3.93196617e-04 3.75861296e-04 2.34494977e-01]
 [3.72709419e-04 4.05804124e-04 3.71133481e-04 2.41161354e-01]] -> [61400.]


In [109]:
# 학습용/테스트용 데이터 생성

# 전체 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.7)
# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size
 
# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])
 
# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

In [110]:
trainX.shape

(294, 5, 4)

In [111]:
trainY.shape

(294, 1)

In [112]:
testX.shape

(126, 5, 4)

In [113]:
trainX = trainX.reshape(294, -1)
trainX.shape

(294, 20)

In [114]:
testX = testX.reshape(126, -1)
testX.shape

(126, 20)

In [115]:
model = Sequential()

model.add(Dense(128,activation = 'relu', input_shape = (20,)))
model.add(Dense(12))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dense(5))
model.add(Dense(5))
model.add(Dense(5))
model.add(Dense(3))
model.add(Dense(1))

In [116]:
# 3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mae'])  
model.fit(trainX, trainY, epochs=100, batch_size=1)  

Epoch 1/100
294/294 [==============================] - 1s 3ms/step - loss: 1189603573.2454 - mae: 28542.7754
Epoch 2/100
294/294 [==============================] - 0s 1ms/step - loss: 40875832.9902 - mae: 5276.7539
Epoch 3/100
294/294 [==============================] - 0s 1ms/step - loss: 39631510.3618 - mae: 5155.4727
Epoch 4/100
294/294 [==============================] - 0s 1ms/step - loss: 36137271.8629 - mae: 4982.2847
Epoch 5/100
294/294 [==============================] - 0s 1ms/step - loss: 34499655.7794 - mae: 4857.6436
Epoch 6/100
294/294 [==============================] - 0s 1ms/step - loss: 31073085.7990 - mae: 4518.1260
Epoch 7/100
294/294 [==============================] - 0s 1ms/step - loss: 30498943.3650 - mae: 4445.6499
Epoch 8/100
294/294 [==============================] - 0s 1ms/step - loss: 29826870.4127 - mae: 4471.2295
Epoch 9/100
294/294 [==============================] - 0s 1ms/step - loss: 27126228.5200 - mae: 4221.1597A: 0s - loss: 24464066.3875 - mae: 401
Epoch

In [117]:
# 4. 평가예측
loss, mse = model.evaluate(testX, testY, batch_size=1)
print('acc : ', mse)

126/126 [==============================] - 0s 1ms/step
acc :  1931.703857421875


In [118]:
y_predict = model.predict(testX, batch_size=1)

# RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
print('RMSE :', RMSE(testY,y_predict))

RMSE : 2289.9430502244327


In [119]:
recent_data = np.array([x_m[ :seq_length ]])


In [120]:
recent_data = recent_data.reshape(1, -1)
recent_data.shape

(1, 20)

In [121]:
x_pred = recent_data
p = model.predict(x_pred, batch_size=1)
print(p)

[[53009.98]]
